In [2]:
using LinearAlgebra, Plots, Printf, SparseArrays, IterativeSolvers
import Base: *
import IterativeSolvers: Adivtype

The conjugate gradient algorithm

In [3]:
function CG(A,b,eps::Float64)
   x = 0.0*b; r = b; p = r; n = 0;
   while norm(r) > eps
        q = A*p
        a = (r'*r)/(p'*q)
        x = x + a*p
        r_old = r
        r = r - a*q
        b = (r'*r)/(r_old'*r_old)
        p = r + b*p 
        n += 1
    end
    @printf("CG took %i iterations",n)
    x
end

CG (generic function with 1 method)

In [4]:
A = SymTridiagonal(fill(4.0,10),fill(-1.0,9));
b = randn(10);

In [5]:
x = CG(A,b,1e-16);
x - A\b |> norm

CG took 10 iterations

2.435541875787129e-16

The conjugate gradient algorithm for general linear functions and inner products

In [6]:
function CG(f,b,⋄,eps::Float64)
   x = 0.0*b; r = b; p = r; n = 0;
   while r⋅r > eps^2
        q = f(p)
        a = (r⋅r)/(p⋅q)
        x = x + a*p
        r_old = r
        r = r - a*q
        b = (r⋅r)/(r_old⋅r_old)
        p = r + b*p
        n += 1
    end
    @printf("CG took %i iterations",n)
    x
end

CG (generic function with 2 methods)

In [7]:
f = x -> A*x
function ⋄(x,y)
    x'*y
end

⋄ (generic function with 1 method)

In [8]:
x = CG(f,b,⋄,1e-16)
A\b - x |> norm

CG took 10 iterations

2.435541875787129e-16

Sylvester matrix equations

$$ A X + X B = C.$$

In [9]:
n = 400;
C = randn(n,n);
A = SymTridiagonal(fill(2.0,n),fill(-1.0,n-1));

In [10]:
X = CG(f,C,⋄,1e-16)

CG took 799 iterations

400×400 Matrix{Float64}:
  -0.203206    2.12517    -8.43964   …    -8.19072   -5.29692   -4.80771
  -1.4887      4.15553   -17.4654        -16.7301   -11.5032    -8.41952
  -2.66948     5.98684   -27.2013        -25.4986   -17.3049   -11.1281
  -4.51895     7.84807   -36.6656        -34.682    -22.3276   -12.3186
  -6.0888      8.54373   -46.3255        -44.7099   -27.5694   -13.3644
  -7.01315     8.89425   -55.5358    …   -55.345    -33.2818   -14.7793
  -8.56978     8.61859   -65.0272        -66.4951   -38.1726   -18.7853
  -9.50273     8.6348    -74.3566        -76.8063   -41.6413   -21.5991
 -11.4824      8.4712    -82.3844        -86.7284   -44.2727   -24.3129
 -13.8688      8.73292   -92.6672        -96.9561   -46.4904   -25.5631
 -17.2006      9.42394  -101.444     …  -106.342    -48.9164   -26.3324
 -20.1915      8.94612  -109.793        -115.73     -51.0029   -27.5647
 -23.5237      8.84003  -117.44         -125.497    -51.9014   -28.614
   ⋮                                 ⋱

Preconditioned CG

In [23]:
# function CG(f,g,b,⋄,eps::Float64)
#    x = 0.0*b; r = b; n = 0; z = g(r);  p = z;
#    while r⋄r > eps^2
#         q = f(p)
#         a = (r⋅z)/(p⋅q)
#         x = x + a*p
#         r_old = r
#         z_old = z
#         r = r - a*q
#         z = g(r)
#         b = (r⋄z)/(r_old⋄z_old)
#         p = z + b*p 
#         n += 1
#     end
#     @printf("CG took %i iterations \n",n)
#     x
# end

function CG(f,g,b,⋄,eps::Float64)
   x = 0.0*b; r = b; n = 0; z = g(r);  p = z;
   for i = 1:1e6
        w = f(p)
        a = (z⋅r)/(p⋅w)
        x = x + a*p
        r_old = r
        z_old = z
        r = r - a*w
        if sqrt(r⋄r) < eps
            break
        end
        z = g(r)
        b = (z⋄r)/(z_old⋄r_old)
        p = z + b*p 
        n += 1
    end
    @printf("CG took %i iterations \n",n)
    x
end

CG (generic function with 3 methods)

Test in trivial case

In [24]:
A = SymTridiagonal(fill(4.0,10),fill(-1.0,9));
b = randn(10);

In [25]:
f = x -> A*x
function ⋄(x,y)
    x'*y
end
g = x -> A\x

#17 (generic function with 1 method)

In [26]:
x = CG(f,g,b,⋄,1e-16)

CG took 1 iterations 


10-element Vector{Float64}:
 -0.17530940439850326
  0.16185035230384456
  0.0926521403623638
 -0.1682210290463347
 -0.18645028033736277
  0.016388342929228934
  0.21531958429876422
  0.41858913617715243
  0.4731197927977504
 -0.07262363138543405

Back to the Sylvester equation

In [40]:
m = 399;
h = 1/(m+1)

0.0025

In [41]:
C = spzeros(m,m);
C[1,:] += rand(m)
C[end,:] += rand(m)
C[:,1] += rand(m)
C[:,end] += rand(m)
A = SymTridiagonal(fill(2.0,m),fill(-1.0,m-1))/h^2;
B = SymTridiagonal(fill(4.0,m),fill(-1.0,m-1))/h^2;

In [42]:
f = X -> A*X + X*A
function ⋄(X,Y)
    h^2*dot(X,Y)
end
g = X -> (.5A+I/h^2)\((.5A+I/h^2)\X')'
#g = X -> (A+2I/h^2)\X

#29 (generic function with 1 method)

In [43]:
@time X = CG(f,C,⋄,h^2);

CG took 863 iterations  1.340420 seconds (265.27 k allocations: 9.229 GiB, 13.64% gc time, 8.55% compilation time)


In [44]:
@time X = CG(f,g,C,⋄,h^2);

CG took 200 iterations 
  1.385950 seconds (957.44 k allocations: 2.697 GiB, 3.20% gc time, 16.61% compilation time)


In [50]:
J = kron(A |> sparse,sparse(I,m,m)) + kron(sparse(I,m,m),A |> sparse);

In [52]:
@time J\rand(m^2)

  0.138234 seconds (69 allocations: 146.706 MiB)


159201-element Vector{Float64}:
 1.0965613500557657e-5
 2.1176567059269073e-5
 2.9199038023567547e-5
 3.7275501401291923e-5
 4.5799400973905554e-5
 5.3270326412215545e-5
 5.894448193978479e-5
 6.679644984107112e-5
 7.415128342133137e-5
 8.022292114823356e-5
 8.677453228166146e-5
 9.231851557019298e-5
 9.645768749640455e-5
 ⋮
 8.948550023171487e-5
 8.437529083380793e-5
 7.860326187851447e-5
 7.224515613264982e-5
 6.66485000782262e-5
 6.047149197624547e-5
 5.390414165189723e-5
 4.6119279464726354e-5
 3.774888388268657e-5
 2.9829173243497542e-5
 2.143583183010199e-5
 1.2084026985391086e-5

In [48]:
J = kron(A |> sparse,sparse(I,m,m),sparse(I,m,m)) + kron(sparse(I,m,m),A |> sparse,sparse(I,m,m)) + kron(sparse(I,m,m),sparse(I,m,m),A |> sparse);

In [ ]:
@time J\rand(m^3)